<a href="https://colab.research.google.com/github/aSafarpoor/SCLFB/blob/main/SCLFB_phase1_one_layer_GNN_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import pickle
import random
import json
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import math
import cv2
from google.colab.patches import cv2_imshow
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
tqdm.pandas()

#go to path

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/MSc/SCLAD/
%mkdir FB15k237
%cd FB15k237

/content/drive/MyDrive/MSc/SCLAD
mkdir: cannot create directory ‘FB15k237’: File exists
/content/drive/MyDrive/MSc/SCLAD/FB15k237


#read data

In [5]:
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/FB15k_mid2name.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/FB15k_mid2description.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/test.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/train.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/valid.txt

In [6]:
!ls

des.p			   FB15k_mid2name.txt  test.pkl  train.txt
FB15k_mid2description.txt  sim.pkl	       test.txt  valid.txt


In [7]:
with open('des.p','rb') as f:
     des = pickle.load(f)

des

,namecode,description,title,text,embedded
0,/m/06rf7,schleswigholstein northernmost sixteen states ...,Schleswig-Holstein,schleswigholstein schleswigholstein northernmo...,"[-0.5489803, 0.16440539, 0.66058505, -0.037717..."
1,/m/0c94fn,gary roger rydstrom american sound designer di...,Gary_Rydstrom,gary_rydstrom gary roger rydstrom american sou...,"[-0.36805353, 1.2219998, 0.22228242, 0.5203364..."
2,/m/016ywr,jeremy john irons english actor receiving clas...,Jeremy_Irons,jeremy_irons jeremy john irons english actor r...,"[-0.8121141, 0.7617216, 0.43965185, -0.0815964..."
3,/m/01yjl,chicago cubs professional baseball team locate...,Chicago_Cubs,chicago_cubs chicago cubs professional basebal...,"[-0.5676567, 0.1930804, -0.13923785, 0.4089947..."
4,/m/02hrh1q,actor person portraying character dramatic com...,Actor-GB,actorgb actor person portraying character dram...,"[0.30152893, 0.9061507, 0.70486355, 0.19482128..."
...,...,...,...,...,...
14899,/m/08jyyk,experimental rock avantgarde rock type music b...,Experimental_rock,experimental_rock experimental rock avantgarde...,"[0.23046279, 0.11413644, 0.11279315, 0.3775108..."
14900,/m/037fqp,marquette university private coeducational jes...,Marquette_University,marquette_university marquette university priv...,"[-0.7175616, 0.98445547, -0.17260364, 0.035183..."
14901,/m/05v954,christopher robin sabat american voice actor a...,Christopher_Sabat,christopher_sabat christopher robin sabat amer...,"[-0.788014, 0.57135695, 0.06602024, 0.16588826..."
14902,/m/03wv2g,mercer university private coeducational univer...,Mercer_University,mercer_university mercer university private co...,"[-0.09629617, 0.82491106, 0.10996592, 0.109891..."


In [8]:
dfname = []
for line in open('FB15k_mid2name.txt', 'r'):
    dfname.append(line.replace('\n','').split('\t'))
dfname = pd.DataFrame(dfname)
dfname = dfname.rename(columns={0: "namecode", 1: "title"})
dfname.head(5)

,namecode,title
0,/m/06rf7,Schleswig-Holstein
1,/m/0c94fn,Gary_Rydstrom
2,/m/016ywr,Jeremy_Irons
3,/m/01yjl,Chicago_Cubs
4,/m/02hrh1q,Actor-GB


In [9]:
train = []
for line in open('train.txt', 'r'):
    train.append(line.replace('\n','').split('\t'))
train = pd.DataFrame(train)
train = train.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
train.head(5)

,namecode1,title,namecode2
0,/m/027rn,/location/country/form_of_government,/m/06cx9
1,/m/017dcd,/tv/tv_program/regular_cast./tv/regular_tv_app...,/m/06v8s0
2,/m/07s9rl0,/media_common/netflix_genre/titles,/m/0170z3
3,/m/01sl1q,/award/award_winner/awards_won./award/award_ho...,/m/044mz_
4,/m/0cnk2q,/soccer/football_team/current_roster./sports/s...,/m/02nzb8


In [10]:
test = []
for line in open('test.txt', 'r'):
    test.append(line.replace('\n','').split('\t'))
test = pd.DataFrame(test)
test = test.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
test.head(5)

,namecode1,title,namecode2
0,/m/08966,/travel/travel_destination/climate./travel/tra...,/m/05lf_
1,/m/01hww_,/music/performance_role/regular_performances./...,/m/01q99h
2,/m/09v3jyg,/film/film/release_date_s./film/film_regional_...,/m/0f8l9c
3,/m/02jx1,/location/location/contains,/m/013t85
4,/m/02jx1,/location/location/contains,/m/0m0bj


In [11]:
valid = []
for line in open('valid.txt', 'r'):
    valid.append(line.replace('\n','').split('\t'))
valid = pd.DataFrame(valid)
valid = valid.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
valid.head(5)

,namecode1,title,namecode2
0,/m/07pd_j,/film/film/genre,/m/02l7c8
1,/m/06wxw,/location/location/time_zones,/m/02fqwt
2,/m/01t94_1,/people/person/spouse_s./people/marriage/type_...,/m/04ztj
3,/m/02xcb6n,/award/award_category/winners./award/award_hon...,/m/04x4s2
4,/m/07f_7h,/film/film/release_date_s./film/film_regional_...,/m/04gzd


In [12]:
def reset_index(df):
    df = df.reset_index()
    del df['index']
    return df

test = reset_index(test)
train = reset_index(train)
valid = reset_index(valid)

#preprocessing

In [13]:
nodes = list(set(des.namecode))

In [14]:
len(nodes)

14904

In [15]:
des.head(5)

,namecode,description,title,text,embedded
0,/m/06rf7,schleswigholstein northernmost sixteen states ...,Schleswig-Holstein,schleswigholstein schleswigholstein northernmo...,"[-0.5489803, 0.16440539, 0.66058505, -0.037717..."
1,/m/0c94fn,gary roger rydstrom american sound designer di...,Gary_Rydstrom,gary_rydstrom gary roger rydstrom american sou...,"[-0.36805353, 1.2219998, 0.22228242, 0.5203364..."
2,/m/016ywr,jeremy john irons english actor receiving clas...,Jeremy_Irons,jeremy_irons jeremy john irons english actor r...,"[-0.8121141, 0.7617216, 0.43965185, -0.0815964..."
3,/m/01yjl,chicago cubs professional baseball team locate...,Chicago_Cubs,chicago_cubs chicago cubs professional basebal...,"[-0.5676567, 0.1930804, -0.13923785, 0.4089947..."
4,/m/02hrh1q,actor person portraying character dramatic com...,Actor-GB,actorgb actor person portraying character dram...,"[0.30152893, 0.9061507, 0.70486355, 0.19482128..."


In [46]:
tempa = des.namecode
tempb = des.embedded
d = {}
for i in range(len(tempa)):
    d[tempa[i]]=[np.array(tempb[i])]    

##train:

In [77]:
num_of_gnn = 25

In [78]:
d2 = {}
for k in list(d.keys()):
    d2[k] = [d[k]]

In [79]:
for _  in range(num_of_gnn):
    print("round:",_+1," all:",num_of_gnn)

    for k in tqdm(range(len(train))):
        row = train.loc[i]
        x = row[0]
        y = row[2]
        d2[x].append(d[y])
        d2[y].append(d[x])

    for k in tqdm(list(d.keys())):    
        d[k]= [np.array(sum(d2[k][0]))/len(d2[k][0])]
        d2[k]=[d[k]]


round: 1  all: 25


100%|██████████| 14904/14904 [00:00<00:00, 155108.70it/s]


round: 2  all: 25


100%|██████████| 14904/14904 [00:00<00:00, 165476.98it/s]


round: 3  all: 25


...

100%|██████████| 14904/14904 [00:00<00:00, 171031.68it/s]


round: 24  all: 25


100%|██████████| 14904/14904 [00:00<00:00, 158502.37it/s]


round: 25  all: 25


100%|██████████| 14904/14904 [00:00<00:00, 152614.72it/s]


In [51]:
print(len(d2[k]))
print(type(d2[k]))
print('-------')
print(len(d[k]))
print(type(d[k]))

1
<class 'list'>
-------
1
<class 'list'>


## test

In [52]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity([d2[k]],[d2[list(d.keys())[0]]])

In [70]:
def similarity(x,y):
    try:
        return cosine_similarity([x],[y])[0][0]
    except:
        return cosine_similarity(x[0],y[0])[0][0]

In [80]:
import warnings
warnings.filterwarnings("ignore")

In [81]:
truecounter = 0
coldstart = 0
for i in tqdm(range(len(test))):
    row = train.loc[i]
    x = row[0]
    y = row[2]

    while(True):
        z = random.randint(0,len(des)-1)
        if  len(test[test.namecode1==x][test.namecode2==des.loc[z][0]])==0:
            try:
                s1 = similarity(d2[x],d2[y])
            except:
                coldstart+=1
                s1 = 0

            try:
                s2 = similarity(d2[x],d2[des.loc[z][0]])
            except:
                coldstart+=1
                s2 = 0
            break
    

    if s1>s2:
        truecounter += 1
    
f = len(test)-truecounter
t = truecounter
print("\n\ncoldstart= ",coldstart)
print(t,f,int(100*t/(t+f)))


100%|██████████| 20466/20466 [01:39<00:00, 206.04it/s]



coldstart=  103
13758 6708 67


num of gnn = 1

    test1: 62% 
    test2: 63%

num of gnn = 5

    test1: 66% 

num of gnn = 15

    test1: 67% 

In [82]:
similarity(d2[x],d2[y])

0.6324043